In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
sys.path.append('/content/gdrive/MyDrive/Project_deep_neural_networks/in-context-learning/src')

!pip install pip==21.2.4

!pip install pytorch
!pip install jupyter==1.0.0
!pip install matplotlib==3.5.2
!pip install numpy==1.22.3
!pip install pandas==1.4.2
!pip install argparse
!pip install scikit-learn==1.0.2
!pip install seaborn==0.11.2
!pip install tqdm==4.64.0
!pip install transformers==4.17.0
!pip install wandb==0.12.11
!pip install xgboost==1.6.1
!pip install protobuf==3.20.1
!pip install munch

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pip-tools 6.13.0 requires pip>=22.2, but you have pip 21.2.4 which is incompatible.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xiix66nr/pytorch_d67831e04e0644e4992c9707696cacee/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xiix66nr/pytorch_d67831e04e0644e4992c9707696cacee/setup.py'"'"';f = getattr(tokenize, '"'"'o

     |████████████████████████████████| 16.8 MB 3.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    distutils: /usr/bin
    sysconfig: /usr/local/bin
    user = False
    home = None
    root = None
    prefix = None
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.22.3 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 1.22.3 which is incompatible.
plotnine 0.12.4 requires matplotlib>=3.6.0, but you have matplotlib 3.5.2 which is i

     |████████████████████████████████| 11.7 MB 3.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
plotnine 0.12.4 requires matplotlib>=3.6.0, but you have matplotlib 3.5.2 which is incompatible.
plotnine 0.12.4 requires numpy>=1.23.0, but you have numpy 1.22.3 which is incompatible.
plotnine 0.12.4 requires pandas>=1.5.0, but you have pandas 1.4.2 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.4.2 which is incompatible.
bigframes 0.15.0 requir

     |████████████████████████████████| 26.5 MB 1.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.15.0 requires pandas>=1.5.0, but you have pandas 1.4.2 which is incompatible.
bigframes 0.15.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 which is incompatible.
     |████████████████████████████████| 292 kB 3.3 MB/s 
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
sys.path.append('/content/gdrive/MyDrive/Project_deep_neural_networks/in-context-learning/src')

Mounted at /content/gdrive


In [2]:
%cd /content
!ls

/content
gdrive	sample_data


In [3]:
from collections import OrderedDict
import re
import os


import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from tqdm.notebook import tqdm
import yaml
import argparse
import uuid
import wandb

from eval import get_run_metrics, read_run_dir, get_model_from_run
from train import main, train
from schema import MainConfig
from plot_utils import basic_plot, collect_results, relevant_model_names

%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set_theme('notebook', 'darkgrid')
palette = sns.color_palette('colorblind')

run_dir = "gdrive/MyDrive/Project_deep_neural_networks/in-context-learning/src/models"

In [4]:
def merge_dicts(dict1, dict2):
    merged_dict = dict1.copy()

    for key, value in dict2.items():
        if key in merged_dict and isinstance(merged_dict[key], dict) and isinstance(value, dict):
            # Recursive merge for nested dictionaries
            merged_dict[key] = merge_dicts(merged_dict[key], value)
        elif key in merged_dict and isinstance(merged_dict[key], list) and isinstance(value, list):
            # Extend lists if both values are lists
            merged_dict[key].extend(value)
        else:
            # Non-dictionary values or new key
            merged_dict[key] = value

    return merged_dict


In [5]:
#use this when continuing training of a model

location="gdrive/MyDrive/Project_deep_neural_networks/in-context-learning/src/" #change this to relevant file structure

model_name="models/decision_tree_and_linear/"

model_run="56a4bee7-75fe-4577-abf4-71a1b8b9a09f/"

run_dir = "gdrive/MyDrive/Project_deep_neural_networks/in-context-learning/src/models"

df = read_run_dir(run_dir)

display(df)

task="decision_tree_and_linear"

run_id="56a4bee7-75fe-4577-abf4-71a1b8b9a09f"

run_path = os.path.join(run_dir, task, run_id)

model, args = get_model_from_run(run_path)

wandb.init(
      dir=args.out_dir,
      project=args.wandb.project,
      entity=args.wandb.entity,
      config=args.__dict__,
      notes=args.wandb.notes,
      name=args.wandb.name,
      resume=True,
)

model=model.cuda().train()

train(model, args, 558900+203700+142255)



,run_id,task,model,kwargs,num_tasks,num_examples,n_dims,n_layer,n_head,run_name
6,56a4bee7-75fe-4577-abf4-71a1b8b9a09f,decision_tree_and_linear,Transformer-small,,-1,-1,20,6,4,decision_tree_and_linear
9,0f7a1f77-7eb9-4429-ac55-33c81e039c57,decision_tree_and_linear_with_labels,Transformer-small,,-1,-1,21,6,4,decision_tree_and_linear_with_labels
3,pretrained,decision_tree,Transformer,depth=4,-1,-1,20,12,8,decision_tree_pretrained
4,c6e5d6d1-f151-414a-8792-dbfc7d116733,decision_tree,Transformer-small,depth=4,-1,-1,20,6,4,decision_tree_standard
8,186cfccb-cd95-4866-acb7-e96bc14966a9,linear_decision_tree_and_NN,Transformer-small,"models=['linear_regression', 'updated_decision...",-1,-1,20,6,4,linear_decision_tree_and_NN
1,pretrained,linear_regression,Transformer,,-1,-1,20,12,8,linear_regression_pretrained
0,pretrained,relu_2nn_regression,Transformer,hidden_layer_size=100,-1,-1,20,12,8,relu_2nn_regression_pretrained
2,pretrained,sparse_linear_regression,Transformer,sparsity=3,-1,-1,20,12,8,sparse_regression_pretrained
7,b2907f75-3b9b-404f-8ba3-c483d9f8abd7,uniform_decision_tree_and_linear,Transformer-small,weights=uniform,-1,-1,20,6,4,uniform_decision_tree_and_linear
5,7110a35b-bb4e-4796-be75-c5fdfd072f1d,updated_decision_tree,Transformer-small,depth=4_mu=0_sigma=1_normalize=False,-1,-1,20,6,4,updated_decision_tree_standard


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


loss 11.204309463500977: 100%|██████████| 46001/46001 [39:39<00:00, 19.33it/s]
